In [20]:
def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name):
    from Bio import SeqIO
   
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if "Period_gene_genomic_sequence_individual_exon" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")

    list_of_files_inside_indiv_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")
    if "temp" not in list_of_files_inside_indiv_exon_folder:
         os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp")

    os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}"')
    os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa"')
    genome = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa", "fasta")
    # genome = SeqIO.parse("/mnt/c/Users/sauba/Downloads/Compressed/ncbi_dataset_7/ncbi_dataset/data/GCA_917051295.2/GCA_917051295.2_ilEreLige1.2_genomic.fna", 'fasta')
    for entries in genome:
        gene_sequence = entries.seq
#         if complement == "1":
#             gene_sequence = gene_sequence.reverse_complement()
        break
    return (gene_sequence)

def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)   

def get_sequence(blast_output_location,genome_location, species, genome_file,annotated_genome_location,annotated_species_name):
    list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
    coordinate_file_name = ''
    for file_names in list_of_files_in_species_folder:
        if file_names.endswith("_coordinates.csv"):
            coordinate_file_name = file_names

    with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
        coordinate_file_lines = temp_file_open.readlines()

    upstream_exon_line =coordinate_file_lines[1].split(",")
    if upstream_exon_line[5] == "Y":
        print("First Exon Error!!")
        assert False

    downstream_exon_line = coordinate_file_lines[-1].split(",")
    if downstream_exon_line[5] == "Y":
        print("Last Exon Error!!")
        assert False

    
    complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]

    if complement == "0":
        gene_start = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
        gene_end = max(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
    
    if complement == "1":
        gene_start = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        gene_end = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))

    
    gene_sequence = get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name)
    print("BB")
    return(gene_sequence,scaffold,gene_start,gene_end )

def get_annotated_genome_name(annotated_genome_location, species):
    list_of_annotated_genomes = os.listdir(annotated_genome_location)
  
    # print(annotated_species)
    for annotated_species in list_of_annotated_genomes:
        if annotated_species.endswith(species):
            return(annotated_species)
             
    if annotated_species_name == '':
        print(f"Error with annotated species name")
        assert False



In [21]:
def make_species_folder(output_location, species):
    list_of_folders = os.listdir(f"{output_location}/5.For Email/")
    if species not in list_of_folders:
        os.mkdir(f"{output_location}/5.For Email/{species}")


In [22]:
import os
import io
import subprocess
genome_location =  "/mnt/f/Genomes_2023-12-26"
# genome_location =  "/mnt/g/Genomes_2023-12-26"
blast_output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
# blast_output_location = "/mnt/j/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
list_of_species = os.listdir(f"{output_location}/2.Final_output")
list_of_species.remove('desktop.ini')
# print(list_of_species)
output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine"

for species in list_of_species:

    annotated_genome_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
    # annotated_genome_location = "/mnt/j/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"

    genome_file = get_genome_file(genome_location,species)
    annotated_species_name = get_annotated_genome_name(annotated_genome_location, species)

    gene_sequence,scaffold, gene_start, gene_end = get_sequence(blast_output_location,genome_location, species, genome_file,
                                                               annotated_genome_location,annotated_species_name)

    print(gene_sequence,scaffold, gene_start, gene_end)
    with open(f"{output_location}/5.For Email_padded/{species}/Genome_fragment.fa",'w') as out_file:
        output = f">{species}_{scaffold}_{gene_start}_{gene_end}\n{gene_sequence}"
        out_file.write(output)

#     local_genomic_fragment_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/"
#     cd_command = f'cd "{local_genomic_fragment_location}"\nmakeblastdb -in gene_sequence_all.fa -dbtype nucl\n'
#         # # os.system(f'{cd_command}')
#     subprocess.run(f'{cd_command}', shell = True, stderr = subprocess.DEVNULL)
#         # # print(mkdb_command)
#         # # os.system(f'{mkdb_command}')
#     # print(gene_sequence)

samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB
... NC_069110.1 7826744 7864955


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB
... OY979626.1 7015875 7050687


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


...

BB
... OV049860.1 10020919 10062359


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB
... JARPMR010000001.1 13904113 13945333


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB
... NC_053208.1 16902847 16934455


In [15]:
import os
output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine"

list_of_species = os.listdir(f"{output_location}/2.Final_output")
list_of_species.remove('desktop.ini')
# print(list_of_species)

for species in list_of_species:
    print(species)
#     print(gene_start)
    make_species_folder(output_location, species)
    output = ''
    list_of_variant_files = os.listdir(f"{output_location}/4.Bed File/{species}/")
    if 'desktop.ini' in list_of_variant_files:
        list_of_variant_files.remove('desktop.ini')
#     start = 0
    
    for variant in list_of_variant_files:    
        with open(f"{output_location}/4.Bed File/{species}/{variant}", 'r') as var_file:
            var_list = var_file.readlines()
        
        if int((var_list[0].split("\t")[1])) <int((var_list[1].split("\t")[1])):
            complement = 0
            gene_start = int((var_list[0].split("\t")[1]))
        else:
            complement = 1
            gene_start = int((var_list[-1].split("\t")[1]))
        output = ''
#         print(var_list)
        for gene_line in  var_list:
            gene_line_split = gene_line.split("\t")
#             print(gene_line, var_list)
            gene_line_split[1] = str(int(gene_line_split[1]) - gene_start)
            gene_line_split[2] = str(int(gene_line_split[2]) - gene_start)
            output += "\t".join(gene_line_split)
        
        print(output)
        with open(f"{output_location}/5.For Email_padded/{species}/{variant}", 'w') as out_file:
            out_file.write(output)
#         print(complement, gene_start)
#         assert False
        

Bicyclus_anynana
NC_069110.1	0	88	Exon_1
NC_069110.1	6419	6490	Exon_2
NC_069110.1	6675	6850	Exon_3
NC_069110.1	8768	8853	Exon_4
NC_069110.1	9197	9265	Exon_5
NC_069110.1	10158	10289	Exon_6
NC_069110.1	12499	12586	Exon_7
NC_069110.1	14707	14910	Exon_8
NC_069110.1	16560	16788	Exon_9
NC_069110.1	19186	19297	Exon_10
NC_069110.1	19699	19818	Exon_11
NC_069110.1	20156	20305	Exon_12
NC_069110.1	21463	21581	Exon_13
NC_069110.1	21886	21990	Exon_14
NC_069110.1	23126	23284	Exon_15
NC_069110.1	24582	24747	Exon_16
NC_069110.1	25545	25680	Exon_17
NC_069110.1	26905	27009	Exon_18
NC_069110.1	28144	28245	Exon_19
NC_069110.1	29199	29321	Exon_20
NC_069110.1	30213	30376	Exon_21
NC_069110.1	32889	33029	Exon_22
NC_069110.1	34492	34680	Exon_23
NC_069110.1	34929	35036	Exon_24
NC_069110.1	36243	36344	Exon_25
NC_069110.1	36423	36572	Exon_26
NC_069110.1	38116	38211	Exon_27

NC_069110.1	0	88	Exon_1
NC_069110.1	6419	6490	Exon_2
NC_069110.1	6675	6850	Exon_3
NC_069110.1	8768	8853	Exon_4
NC_069110.1	9197	9265	Exon_5
NC

JACTOM010000032.1	36854	36942	Exon_1
JACTOM010000032.1	35847	35930	Exon_2
JACTOM010000032.1	35280	35455	Exon_3
JACTOM010000032.1	32029	32123	Exon_4
JACTOM010000032.1	30712	30780	Exon_5
JACTOM010000032.1	28746	28877	Exon_6
JACTOM010000032.1	28273	28348	Exon_7
JACTOM010000032.1	26958	27161	Exon_8
JACTOM010000032.1	26100	26328	Exon_9
JACTOM010000032.1	24002	24113	Exon_10
JACTOM010000032.1	23452	23568	Exon_11
JACTOM010000032.1	22122	22286	Exon_12
JACTOM010000032.1	21593	21711	Exon_13
JACTOM010000032.1	20053	20151	Exon_14
JACTOM010000032.1	18949	19083	Exon_15
JACTOM010000032.1	17018	17192	Exon_16
JACTOM010000032.1	15337	15478	Exon_17
JACTOM010000032.1	10480	10584	Exon_18
JACTOM010000032.1	9480	9578	Exon_19
JACTOM010000032.1	7126	7254	Exon_20
JACTOM010000032.1	6319	6473	Exon_21
JACTOM010000032.1	2677	2838	Exon_23
JACTOM010000032.1	2189	2308	Exon_24
JACTOM010000032.1	1689	1790	Exon_25
JACTOM010000032.1	874	1014	Exon_26
JACTOM010000032.1	0	101	Exon_27

JACTOM010000032.1	36854	36942	Exon_1
JACT

OW121737.1	31362	31450	Exon_1
OW121737.1	29633	29716	Exon_2
OW121737.1	29112	29287	Exon_3
OW121737.1	28408	28496	Exon_4
OW121737.1	27754	27825	Exon_5
OW121737.1	16115	16246	Exon_6
OW121737.1	15606	15681	Exon_7
OW121737.1	14745	14948	Exon_8
OW121737.1	13610	13838	Exon_9
OW121737.1	11205	11316	Exon_10
OW121737.1	10998	11117	Exon_11
OW121737.1	10202	10351	Exon_12
OW121737.1	9675	9793	Exon_13
OW121737.1	9455	9562	Exon_14
OW121737.1	8976	9122	Exon_15
OW121737.1	7825	7999	Exon_16
OW121737.1	6986	7067	Exon_17
OW121737.1	5901	5975	Exon_19
OW121737.1	5050	5220	Exon_20
OW121737.1	4627	4781	Exon_21
OW121737.1	3350	3478	Exon_22
OW121737.1	2406	2594	Exon_23
OW121737.1	1863	1976	Exon_24
OW121737.1	971	1072	Exon_25
OW121737.1	584	721	Exon_26
OW121737.1	0	95	Exon_27

OW121737.1	31362	31450	Exon_1
OW121737.1	29633	29716	Exon_2
OW121737.1	29112	29287	Exon_3
OW121737.1	28408	28496	Exon_4
OW121737.1	27754	27825	Exon_5
OW121737.1	16115	16246	Exon_6
OW121737.1	15606	15681	Exon_7
OW121737.1	14745	14948	Exon_

NC_060058.1	0	88	Exon_1
NC_060058.1	1874	1957	Exon_2
NC_060058.1	2193	2368	Exon_3
NC_060058.1	3070	3158	Exon_4
NC_060058.1	3746	3820	Exon_5
NC_060058.1	4023	4154	Exon_6
NC_060058.1	4662	4737	Exon_7
NC_060058.1	6111	6314	Exon_8
NC_060058.1	7233	7461	Exon_9
NC_060058.1	8934	9045	Exon_10
NC_060058.1	9585	9704	Exon_11
NC_060058.1	10000	10149	Exon_12
NC_060058.1	11045	11163	Exon_13
NC_060058.1	11368	11472	Exon_14
NC_060058.1	12452	12595	Exon_15
NC_060058.1	12681	12852	Exon_16
NC_060058.1	13018	13153	Exon_17
NC_060058.1	13755	13856	Exon_19
NC_060058.1	14259	14429	Exon_20
NC_060058.1	15141	15295	Exon_21
NC_060058.1	19170	19313	Exon_22
NC_060058.1	28492	28671	Exon_23
NC_060058.1	29092	29205	Exon_24
NC_060058.1	30005	30106	Exon_25
NC_060058.1	30657	30794	Exon_26
NC_060058.1	31250	31351	Exon_27

NC_060058.1	0	88	Exon_1
NC_060058.1	1874	1957	Exon_2
NC_060058.1	2193	2368	Exon_3
NC_060058.1	3070	3158	Exon_4
NC_060058.1	3746	3820	Exon_5
NC_060058.1	4023	4154	Exon_6
NC_060058.1	4662	4737	Exon_7
NC_06

JARPMR010000001.1	0	88	Exon_1
JARPMR010000001.1	3642	3725	Exon_2
JARPMR010000001.1	3981	4156	Exon_3
JARPMR010000001.1	5402	5490	Exon_4
JARPMR010000001.1	5809	5883	Exon_5
JARPMR010000001.1	13001	13132	Exon_6
JARPMR010000001.1	13792	13867	Exon_7
JARPMR010000001.1	14795	14998	Exon_8
JARPMR010000001.1	16873	17101	Exon_9
JARPMR010000001.1	19704	19815	Exon_10
JARPMR010000001.1	19909	20028	Exon_11
JARPMR010000001.1	21145	21294	Exon_12
JARPMR010000001.1	22396	22514	Exon_13
JARPMR010000001.1	22637	22729	Exon_14
JARPMR010000001.1	25805	25957	Exon_15
JARPMR010000001.1	26060	26234	Exon_16
JARPMR010000001.1	26643	26781	Exon_17
JARPMR010000001.1	32645	32749	Exon_18
JARPMR010000001.1	33091	33195	Exon_19
JARPMR010000001.1	34520	34693	Exon_20
JARPMR010000001.1	35069	35223	Exon_21
JARPMR010000001.1	38905	39093	Exon_23
JARPMR010000001.1	39588	39701	Exon_24
JARPMR010000001.1	40064	40165	Exon_25
JARPMR010000001.1	40438	40575	Exon_26
JARPMR010000001.1	41125	41220	Exon_27

JARPMR010000001.1	0	88	Exon_1
JARPM